In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기
train = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv')
test = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')


# EDA및 전처리

데이터 살펴보기

In [ ]:
print("Train data shape:", train.shape)
print("Test data shape:", test.shape)
train.head()

In [ ]:
train.info()

각종 id와 matchType만 문자열변수  
나머지는 정수형 변수  
id 속성들은 삭제, matchType는이후에 처리

In [ ]:
train.describe()


결측치 처리

In [ ]:
print("Missing values in train data:\n", train.isnull().sum().sort_values(ascending=False).head(10))
print("Missing values in test data:\n", test.isnull().sum().sort_values(ascending=False).head(10))

In [ ]:
train = train.dropna(subset=['winPlacePerc'])

In [ ]:
print("Missing values in train data:\n", train.isnull().sum().sort_values(ascending=False).head(10))

### 이후 EDA및 전처리에서는 베이스라인 모델의 성능을 기준으로 결정함

### 타깃값의 분포

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(train['winPlacePerc'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of winPlacePerc')
plt.xlabel('winPlacePerc')
plt.ylabel('Frequency')
plt.show()


최하위 성적을 기록한 데이터가 많음 [0.1,1.0]의 값들은 상대적으로 균일한 값을 보임

### 3가지 랭크 포인트 변수들 살펴보기

In [ ]:
numeric_train = train.select_dtypes(include=[np.number])

# 세 가지 변수의 분포를 시각화
plt.figure(figsize=(15, 5))

# WinPoints 분포
plt.subplot(1, 3, 1)
plt.hist(numeric_train['winPoints'], bins=30, color='skyblue', edgecolor='black')
plt.title("Distribution of WinPoints")
plt.xlabel("WinPoints")
plt.ylabel("Frequency")

# KillPoints 분포
plt.subplot(1, 3, 2)
plt.hist(numeric_train['killPoints'], bins=30, color='salmon', edgecolor='black')
plt.title("Distribution of KillPoints")
plt.xlabel("KillPoints")
plt.ylabel("Frequency")

# RankPoints 분포
plt.subplot(1, 3, 3)
plt.hist(numeric_train['rankPoints'], bins=30, color='lightgreen', edgecolor='black')
plt.title("Distribution of RankPoints")
plt.xlabel("RankPoints")
plt.ylabel("Frequency")

plt.tight_layout()
plt.show()

대부분의 플레이어가 point가 0인상태여서 상관계수가 낮게 나옴

### 주요 변수들의 분포 살펴보기

In [ ]:
# 주요 수치형 변수 분포 확인
num_columns = ['DBNOs', 'damageDealt', 'kills', 'killStreaks', 'longestKill', 'vehicleDestroys']

train[num_columns].hist(bins=30, figsize=(15, 10), color='lightgreen', edgecolor='black')
plt.suptitle('Distribution of Major Numerical Features')
plt.show()


In [ ]:
num_columns = ['rideDistance', 'swimDistance', 'walkDistance']

train[num_columns].hist(bins=30, figsize=(15, 10), color='lightgreen', edgecolor='black')
plt.suptitle('Distribution of Major Numerical Features')
plt.show()

In [ ]:
num_columns = ['boosts',  'heals', 'weaponsAcquired']

train[num_columns].hist(bins=30, figsize=(15, 10), color='lightgreen', edgecolor='black')
plt.suptitle('Distribution of Major Numerical Features')
plt.show()

In [ ]:
num_columns = ['assists',  'revives', 'teamKills']

train[num_columns].hist(bins=30, figsize=(15, 10), color='lightgreen', edgecolor='black')
plt.suptitle('Distribution of Major Numerical Features')
plt.show()

대부분 정규분포 형태를 띄지않고 한쪽으로 치우친 모습

### 매치타입 확인하기

In [ ]:
# matchType 분포 확인
plt.figure(figsize=(12, 6))
train['matchType'].value_counts().plot(kind='bar', color='lightblue', edgecolor='black')
plt.title('Distribution of matchType')
plt.xlabel('matchType')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()


FPP(1인칭)와 TPP(3인칭)은 시점의 차이  
solo, duo, squad 의 매치타입으로 이루어져있음  

### 주요변수와 타깃값과의 상관관계

In [ ]:
import seaborn as sns

# 주요 성과 변수와 winPlacePerc 간의 상관관계
plt.figure(figsize=(12, 8))
sns.scatterplot(data=train, x='kills', y='winPlacePerc', alpha=0.3, color='red')
plt.title('Kills vs winPlacePerc')
plt.show()

sns.scatterplot(data=train, x='damageDealt', y='winPlacePerc', alpha=0.3, color='blue')
plt.title('DamageDealt vs winPlacePerc')
plt.show()

sns.scatterplot(data=train, x='walkDistance', y='winPlacePerc', alpha=0.3, color='green')
plt.title('WalkDistance vs winPlacePerc')
plt.show()


킬, 이동, 대미지를 넣지않고 높은 순위를 차지하는 경우도 있지만  
각각의 변수가 증가함에따라 타깃변수도 증가하는 추세

In [ ]:
kill_related = ['kills', 'killStreaks', 'headshotKills', 'DBNOs']
sns.pairplot(train[kill_related], diag_kind='kde')
plt.suptitle("Relationships between Kill-related Variables")
plt.show()


기절, 헤드샷킬, 연속킬 역시 킬횟수와 상관관계가 나옴.

### 아이템사용과 타깃값간의 상관관계

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=train, x='boosts', y='winPlacePerc', alpha=0.4, color='orange')
plt.title('Boosts vs winPlacePerc')
plt.show()

sns.scatterplot(data=train, x='heals', y='winPlacePerc', alpha=0.4, color='teal')
plt.title('Heals vs winPlacePerc')
plt.show()


부스트 아이템과 힐링 아이템 모두 타깃값과 상관관계 존재

### 팀플레이지수와 타깃값간의 상관관계

In [ ]:
team_related = ['revives', 'teamKills', 'assists', 'winPlacePerc']
sns.pairplot(train[team_related], diag_kind='kde')
plt.suptitle("Relationships between Team-play Variables and winPlacePerc")
plt.show()


부활과 어시스트 속성은 타깃값과 상관관계가 존재해보임  
팀킬 지수는 두드러지는 패턴X  
But, 팀킬 지수와 부활&어시스트 속성은 음의 상관관계 존재 (의도적으로 팀킬하는 플레이어들 존재, 해당 플레이어들은 팀플레이 안하므로 팀플레이 지수는 낮을것)

### 이동지수와 타깃값간의 상관관계

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=train, x='walkDistance', y='winPlacePerc', alpha=0.4, color='blue')
plt.title('WalkDistance vs winPlacePerc')
plt.show()

sns.scatterplot(data=train, x='rideDistance', y='winPlacePerc', alpha=0.4, color='purple')
plt.title('RideDistance vs winPlacePerc')
plt.show()

sns.scatterplot(data=train, x='swimDistance', y='winPlacePerc', alpha=0.4, color='green')
plt.title('SwimDistance vs winPlacePerc')
plt.show()


3가지의 이동 방법모두 패턴을 보임

### 상/하위 그룹별 특징분석

In [ ]:
top_10 = train[train['winPlacePerc'] >= 0.9]
bottom_10 = train[train['winPlacePerc'] <= 0.1]

# 상위 10%, 하위 10% 비교
features = ['kills', 'damageDealt', 'walkDistance', 'boosts', 'heals', 'revives']
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
for i, feature in enumerate(features):
    sns.kdeplot(top_10[feature], ax=axes[i//3, i%3], color='blue', label='Top 10%')
    sns.kdeplot(bottom_10[feature], ax=axes[i//3, i%3], color='red', label='Bottom 10%')
    axes[i//3, i%3].set_title(f"{feature} Distribution for Top 10% vs Bottom 10%")
    axes[i//3, i%3].legend()
plt.tight_layout()
plt.show()


하위권의 경우 모든 지수에서 0과 가까운 값을 보임  
상위권의 경우 하위권보다 우측에서 분포를 이룸

### 매치타입별 플레이속성

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# matchType 데이터에서 상위 6개 항목 선택
top_6_match_types = train['matchType'].value_counts().nlargest(6).index
match_type_means_log_top6 = np.log1p(train[train['matchType'].isin(top_6_match_types)]
                                     .groupby('matchType')[['kills', 'damageDealt', 'walkDistance', 'weaponsAcquired', 'boosts']].mean())

# 시각화
fig, ax = plt.subplots(figsize=(15, 8))
match_type_means_log_top6.plot(kind='bar', ax=ax, width=0.8)

# 시각적 개선
ax.set_title("Log-Transformed Average Features by Top 6 Match Types")
ax.set_xlabel("Match Type")
ax.set_ylabel("Log(Average Value)")
ax.legend(title="Feature")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show() 

In [ ]:
features = ['kills', 'damageDealt', 'walkDistance', 'boosts', 'weaponsAcquired']
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for i, feature in enumerate(features):
    ax = axes[i // 3, i % 3]
    match_type_means_log_top6[feature].plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
    ax.set_title(f"Average {feature} by Match Type (Log-Transformed)")
    ax.set_xlabel("Match Type")
    ax.set_ylabel(f"Log(Average {feature})")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()


매치타입별로 다른 속성값은 비슷하게 나타나지만  
킬, 대미지 속성의 경우 차이가남  
-> 매치타입별로 다른 플레이스타일이 나타남 / 다른 전략 수립 필요


# 3. feature engineering & 데이터 전처리

### 수치형 변수들 로그변환

In [ ]:
numeric_columns = train.select_dtypes(include=[np.number]).columns

# train 데이터 로그 변환
#train_log_transformed = train.copy()
#for col in numeric_columns:
#    train[col] = np.log1p(train_log_transformed[col])
# test 데이터 로그 변환
#test_log_transformed = test.copy()
#for col in numeric_columns:
#    if col in test_log_transformed.columns:
#        test[col] = np.log1p(test_log_transformed[col])

In [ ]:
# 로그 변환된 후 몇 가지 주요 수치형 변수 선택
selected_columns = ['kills', 'damageDealt', 'walkDistance', 'heals', 'winPlacePerc']

# 각 변수의 분포 시각화 (train 데이터만)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(selected_columns):
    ax = axes[i]
    ax.hist(train[col], bins=30, color='skyblue', edgecolor='black')
    ax.set_title(f'Log-Transformed Distribution of {col} (Train)')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')

# 빈 그래프 제거
fig.delaxes(axes[-1])
plt.tight_layout()
plt.show()

변환 전:  0.1455938048336897  
변환 후:  0.1455938048336897

### 헤드샷 비율 속성 생성

In [ ]:
train['headshot_rate'] = train['headshotKills'] / (train['kills'] + 1)
test['headshot_rate'] = test['headshotKills'] / (test['kills'] + 1)


In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=train, x='headshot_rate', y='winPlacePerc', alpha=0.5, color='blue')
plt.title("Relationship between Headshot Rate and winPlacePerc")
plt.xlabel("Headshot Rate")
plt.ylabel("winPlacePerc")
plt.show()

### 총 이동거리

In [ ]:
train['total_distance'] = train['walkDistance'] + train['rideDistance'] + train['swimDistance']
test['total_distance'] = test['walkDistance'] + test['rideDistance'] + test['swimDistance']

# 기존 거리 관련 속성 삭제
train = train.drop(columns=['walkDistance', 'rideDistance', 'swimDistance'])
test = test.drop(columns=['walkDistance', 'rideDistance', 'swimDistance'])

train.head()

### 아이템 사용 지표

In [ ]:
train['healsAndBoosts'] = train['heals']+train['boosts']
test['healsAndBoosts'] = test['heals']+test['boosts']
#train = train.drop(columns=['boosts', 'heals', 'weaponsAcquired'])
#test = test.drop(columns=['boosts', 'heals', 'weaponsAcquired'])
train.head()

### 팀원 멤버의수 속성추가

In [ ]:
train['team_size'] = train.groupby(['matchId', 'groupId'])['Id'].transform('count')
test['team_size'] = test.groupby(['matchId', 'groupId'])['Id'].transform('count')

# 같은 팀에 속한 팀원 수 확인
train[['matchId', 'groupId', 'Id', 'team_size']].drop_duplicates().head(10)
test[['matchId', 'groupId', 'Id', 'team_size']].drop_duplicates().head(10)


### ID 속성들 삭제

In [ ]:
#train = train.drop(columns=['matchId', 'groupId'])
#test = test.drop(columns=['matchId', 'groupId'])

train

### killPlace와winPlacePerc의 관계

In [ ]:
# kills가 0인 그룹에서 killPlace와 winPlacePerc의 관계를 시각화
sample_data = train[train['kills'] == 0]

# killPlace와 winPlacePerc의 관계 시각화
plt.figure(figsize=(10, 6))
plt.scatter(sample_data['killPlace'], sample_data['winPlacePerc'], alpha=0.3, color='blue')
plt.title("Relationship between killPlace and winPlacePerc (kills = 0)")
plt.xlabel("killPlace")
plt.ylabel("winPlacePerc")
plt.gca().invert_xaxis()  # killPlace 순서가 낮을수록 높은 순위이므로 축 반전
plt.show()

In [ ]:
#train = train.drop(columns=['killPlace'])
#train

-> 같은 kill을 기록해도 최종 순위가 높으면 높은 kill순위로 기록됨  
해당 속성은 타깃 데이터가  유출되어 적용되는 속성으로 삭제 처리


### numGroups & maxPlace

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(train['numGroups'], train['maxPlace'], alpha=0.5, color='blue')
plt.title('Relationship between numGroups and maxPlace')
plt.xlabel('numGroups')
plt.ylabel('maxPlace')
plt.show()

# 두 속성 간의 상관관계 계산
correlation = train[['numGroups', 'maxPlace']].corr().iloc[0, 1]
print(f"Correlation between numGroups and maxPlace: {correlation:.4f}")

In [ ]:
#train = train.drop(columns=['numGroups'])
#test = test.drop(columns=['numGroups'])


### 매치 정보 속성들 삭제

In [ ]:
#train = train.drop(columns=['maxPlace', 'matchDuration'])
#test = test.drop(columns=['maxPlace', 'matchDuration'])


### 랭크 point 속성 삭제

In [ ]:
#train = train.drop(columns=['winPoints', 'killPoints', 'rankPoints'])
#test = test.drop(columns=['winPoints', 'killPoints', 'rankPoints'])


In [ ]:
test.info()

### Match Type별로 데이터 구분하여 학습 진행하기  
solo, duo, squad는 플레이 특성이 다르기 때문에 다른 데이터로 분류


In [ ]:
plt.figure(figsize=(10, 6))
train['matchType'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title("Distribution of Match Types")
plt.xlabel("Match Type")
plt.ylabel("Frequency")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 상위 6개의 matchType 항목을 가진 데이터만 남기기
top_6_match_types_list = train['matchType'].value_counts().nlargest(6).index
train_top_6_match_types = train[train['matchType'].isin(top_6_match_types_list)]
test_top_6_match_types = test[test['matchType'].isin(top_6_match_types_list)]


# 결과 확인
print(train_top_6_match_types['matchType'].value_counts())

속성 추가 전: 0.1487116553644863  
속성 추가 후: 0.1455938048336897  
-> 속성 추가 후 성능 향상

### matchType별로 데이터셋 분리하기

In [ ]:
#데이터 전처리 과정이 남았기에 마지막에 분리 진행

# Solo 데이터
solo_types = ['solo', 'solo-fpp']
solo_train = train_top_6_match_types[train_top_6_match_types['matchType'].isin(solo_types)]
solo_test = test_top_6_match_types[test_top_6_match_types['matchType'].isin(solo_types)]

# Duo 데이터
duo_types = ['duo', 'duo-fpp']
duo_train = train_top_6_match_types[train_top_6_match_types['matchType'].isin(duo_types)]
duo_test = test_top_6_match_types[test_top_6_match_types['matchType'].isin(duo_types)]

# Squad 데이터
squad_types = ['squad', 'squad-fpp']
squad_train = train_top_6_match_types[train_top_6_match_types['matchType'].isin(squad_types)]
squad_test = test_top_6_match_types[test_top_6_match_types['matchType'].isin(squad_types)]

# 결과 확인
print("Solo 데이터 개수:", len(solo_train))
print("Duo 데이터 개수:", len(duo_train))
print("Squad 데이터 개수:", len(squad_train))

# Solo 테스트 데이터 (matchType에 'solo' 포함)
#solo_test = test[test['matchType'].str.contains('solo')]

# Duo 테스트 데이터 (matchType에 'duo' 포함)
#duo_test = test[test['matchType'].str.contains('duo')]

# Squad 테스트 데이터 (solo, duo에 포함되지 않는 나머지)
#squad_test = test[~test['matchType'].str.contains('solo|duo')]

# 결과 확인
print("Solo 테스트 데이터 개수:", len(solo_test))
print("Duo 테스트 데이터 개수:", len(duo_test))
print("Squad 테스트 데이터 개수:", len(squad_test))

In [ ]:
duo_train

killPlace: 매치내 킬 기반 순위이기에 (-)방향으로 상관관계 나타남  
matchDuration: 해당 매치의 지속시간 (플레이어와 무관)  
3개의 points속성: 해당플레이어에게 부여된 점수  
winpoints: 승리기반의 외부 점수
killpoints: 킬수 기반의 외부 점수  
rankpoints: 승리+킬수 기반의 외부점수


walkDistance가 가장 강한 상관관게를 보임 -> 숨어서 다른 생존자들이 탈락하길 기다리는 전략보다, 적극적으로 움직이는 전략이 좋음  
boosts, weaponsacquires 역시 강한 상관관계를 보임 -> 적극적으로 움직이며 아이템을 얻는 것이 좋음


결측치 제거  
KillPlace 속성 삭제  
MatchTypr별로 데이터 분리  
팀 멤버수 속성 추가


# 4. 베이스 라인 모델

모든 매치타입 한번에 입력 

In [ ]:
#from sklearn.model_selection import train_test_split


#X = train.drop(columns=['winPlacePerc'])
#y = train['winPlacePerc']

#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_squared_error

# 선형 회귀 모델 학습
#lin_reg = LinearRegression()
#lin_reg.fit(X_train, y_train)

# 검증 세트 예측 및 평가 (RMSE 계산)
#y_pred = lin_reg.predict(X_val)
#rmse = np.sqrt(mean_squared_error(y_val, y_pred))
#print("Baseline Linear Regression RMSE:", rmse)


매치타입별 학습

In [ ]:
train

In [ ]:
def calculate_weighted_rmse(y_true_solo, y_pred_solo, y_true_duo, y_pred_duo, y_true_squad, y_pred_squad):
    # 각 검증 세트의 RMSE 계산
    rmse_solo = mean_squared_error(y_true_solo, y_pred_solo, squared=False)
    rmse_duo = mean_squared_error(y_true_duo, y_pred_duo, squared=False)
    rmse_squad = mean_squared_error(y_true_squad, y_pred_squad, squared=False)
    
    # 각 매치타입 검증 세트의 크기 계산
    n_solo = len(y_true_solo)
    n_duo = len(y_true_duo)
    n_squad = len(y_true_squad)
    total_n = n_solo + n_duo + n_squad

    # 가중 평균 RMSE 계산
    weighted_rmse = (rmse_solo * n_solo + rmse_duo * n_duo + rmse_squad * n_squad) / total_n
    return weighted_rmse

## 성능 개선

1. others 그룹 추가

In [ ]:
# train과 test 데이터의 matchType 나머지 값을 '기타'로 변환
top_6_match_types_list = train['matchType'].value_counts().nlargest(6).index
# Train 데이터에서 탑6 제외한 카테고리 분포 확인
train_other_match_types = train[~train['matchType'].isin(top_6_match_types_list)]
print("Train 데이터 탑6 제외한 matchType 분포:")
print(train_other_match_types['matchType'].value_counts())
# Test 데이터에서 탑6 제외한 카테고리 분포 확인
test_other_match_types = test[~test['matchType'].isin(top_6_match_types_list)]
print("\nTest 데이터 탑6 제외한 matchType 분포:")
print(test_other_match_types['matchType'].value_counts())

상위 6개 매치 타입말고도 다른 매치 타입을 고려하지 않기엔 그 수가 너무 많았음

In [ ]:
# train과 test 데이터의 matchType 나머지 값을 '기타'로 변환
top_6_match_types_list = train['matchType'].value_counts().nlargest(6).index

train['matchType'] = train['matchType'].apply(lambda x: x if x in top_6_match_types_list else 'others')
test['matchType'] = test['matchType'].apply(lambda x: x if x in top_6_match_types_list else 'others')

# 변환 후 결과 확인
print("Train 데이터 matchType 카테고리 분포:")
print(train['matchType'].value_counts())
print("\nTest 데이터 matchType 카테고리 분포:")
print(test['matchType'].value_counts())

In [ ]:
# 탑6 카테고리 추출
top_6_match_types_list = train['matchType'].value_counts().nlargest(6).index

# train_top_6_match_types와 test_top_6_match_types 생성
train_top_6_match_types = train[train['matchType'].isin(top_6_match_types_list)].copy()
test_top_6_match_types = test[test['matchType'].isin(top_6_match_types_list)].copy()

print("탑6 카테고리를 포함한 train 데이터 크기:", train_top_6_match_types.shape)
print("탑6 카테고리를 포함한 test 데이터 크기:", test_top_6_match_types.shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Others 데이터 분리
others_train = train_top_6_match_types[~train_top_6_match_types['matchType'].isin(['solo', 'duo', 'squad'])]
others_test = test_top_6_match_types[~test_top_6_match_types['matchType'].isin(['solo', 'duo', 'squad'])]

X_others = others_train.drop(columns=['Id', 'winPlacePerc', 'matchType'])
y_others = others_train['winPlacePerc']

# Others 데이터 검증 세트 분리
X_others_train, X_others_val, y_others_train, y_others_val = train_test_split(X_others, y_others, test_size=0.2, random_state=42)


In [ ]:
train

In [ ]:
test

3. 부스트와 힐의 가중치

In [ ]:
data = train.copy()
data = data[data['heals'] < data['heals'].quantile(0.99)]
data = data[data['boosts'] < data['boosts'].quantile(0.99)]

f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='heals',y='winPlacePerc',data=data,color='lime')
sns.pointplot(x='boosts',y='winPlacePerc',data=data,color='blue')
plt.text(4,0.65,'Heals',color='lime',fontsize = 17,style = 'italic')
plt.text(4,0.8,'Boosts',color='blue',fontsize = 17,style = 'italic')
plt.xlabel('Number of heal/boost items',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Heals vs Boosts',fontsize = 20,color='blue')
plt.grid()
plt.show()

부스트에 가중치를 좀 더 주자

In [ ]:
train['healsAndBoosts'] = train['heals']+2*train['boosts']
test['healsAndBoosts'] = test['heals']+2*test['boosts']

In [ ]:
healsAndBoosts = train[['heals','winPlacePerc','boosts']]
healsAndBoosts['healsAndBoosts'] = train['healsAndBoosts']
healsAndBoosts.corr()

In [ ]:
train=train.drop(columns={'heals', 'boosts'})

In [ ]:
test=test.drop(columns={'heals', 'boosts'})

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train['healsAndBoostsPerWalkDistance'] = train['healsAndBoosts']/(train['total_distance']+1) #The +1 is to avoid infinity.
train['healsAndBoostsPerWalkDistance'].fillna(0, inplace=True)

In [ ]:
test['healsAndBoostsPerWalkDistance'] = test['healsAndBoosts']/(test['total_distance']+1) #The +1 is to avoid infinity.
test['healsAndBoostsPerWalkDistance'].fillna(0, inplace=True)

In [ ]:
train['killsPerWalkDistance'] = train['kills']/(train['total_distance']+1) #The +1 is to avoid infinity, because there are entries where kills>0 and walkDistance=0. Strange.
test['killsPerWalkDistance'] = test['kills']/(test['total_distance']+1) #The +1 is to avoid infinity, because there are entries where kills>0 and walkDistance=0. Strange.
train['killsPerWalkDistance'].fillna(0, inplace=True)
train[['kills', 'total_distance', 'killsPerWalkDistance', 'winPlacePerc']].sort_values(by='killsPerWalkDistance').tail(10)


In [ ]:
train[['kills','total_distance', 'killsPerWalkDistance', 'winPlacePerc']].sort_values(by='kills').tail(10)

4. 매치 내 사람 수

In [ ]:
# playersJoined
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')
test['playersJoined'] = test.groupby('matchId')['matchId'].transform('count')

In [ ]:
train['playersJoined']

In [ ]:
train

헤드샷 비율?

In [ ]:
headshot = train[['headshotKills','winPlacePerc','headshot_rate']]
headshot.corr()

In [ ]:
train

트리 기반
permutation importance” or “Mean Decrease Accuracy (MDA) -> 특성 선택
SHAP values


In [ ]:
train

In [ ]:
test

In [ ]:
#numeric_columns = train.select_dtypes(include=[np.number]).columns


In [ ]:
#numeric_columns 

In [ ]:
# 주요 수치형 변수 분포 확인
num_columns = ['killPlace', 'matchDuration', 'maxPlace', 'numGroups', 'team_size', 'playersJoined']

train[num_columns].hist(bins=30, figsize=(15, 10), color='lightgreen', edgecolor='black')
plt.suptitle('Distribution of Major Numerical Features')
plt.show()


In [ ]:
#numeric_columns = numeric_columns.drop(labels=['winPlacePerc', 'killPlace', 'matchDuration', 'maxPlace', 'numGroups', 'team_size', 'playersJoined'])

In [ ]:
#numeric_columns

# train 데이터 로그 변환
train_log_transformed = train.copy()
for col in numeric_columns:
    train[col] = np.log1p(train_log_transformed[col])
# test 데이터 로그 변환
test_log_transformed = test.copy()
for col in numeric_columns:
    if col in test_log_transformed.columns:
        test[col] = np.log1p(test_log_transformed[col])

모델링

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
# 데이터 처리 및 모델 평가
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# 모델 및 하이퍼파라미터 최적화
import lightgbm as lgb
import optuna

# 경고 메시지 무시 (선택 사항)
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from sklearn.preprocessing import StandardScaler

def prepare_data(train_type, test_type):
    """
    주어진 매치 타입 데이터프레임에서 학습 및 테스트 데이터를 준비하고 표준화하는 함수.

    Parameters:
        train_type (pd.DataFrame): 학습 데이터.
        test_type (pd.DataFrame): 테스트 데이터.

    Returns:
        X_train_scaled (pd.DataFrame): 표준화된 학습 피처 데이터.
        y_train (pd.Series): 학습 타겟 데이터.
        X_test_scaled (pd.DataFrame): 표준화된 테스트 피처 데이터.
    """
    # 피처와 타겟 분리
    X_train = train_type.drop(columns=['Id', 'groupId', 'matchId', 'winPlacePerc', 'matchType'])
    y_train = train_type['winPlacePerc']
    X_test = test_type.drop(columns=['Id', 'groupId', 'matchId', 'matchType'])

    # 표준화
    scaler = StandardScaler()

    # 훈련 데이터를 기준으로 스케일러 학습 및 변환
    X_train_scaled = scaler.fit_transform(X_train)
    # 테스트 데이터는 훈련 데이터의 스케일러를 사용해 변환
    X_test_scaled = scaler.transform(X_test)

    # NumPy 배열을 DataFrame으로 변환 (원래의 컬럼 이름 및 인덱스 유지)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

    return X_train_scaled, y_train, X_test_scaled


In [ ]:
def train_lgb_with_feature_selection(X, y):
    """
    LightGBM 모델을 학습하고 최적의 하이퍼파라미터를 찾는 함수 (특성 선택 포함).
    """
    feature_names = X.columns.tolist()  # 특성 이름 리스트

    def objective(trial):
        # 특성 선택
        selected_indices = [
            i for i in range(len(feature_names)) if trial.suggest_categorical(f"use_feature_{i}", [True, False])
        ]
        selected_features = [feature_names[i] for i in selected_indices]
        
        if len(selected_features) == 0:  # 특성이 하나도 선택되지 않은 경우 방지
            return float("inf")

        # 하이퍼파라미터 정의
        param = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 150),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
            'verbosity': -1,  # 오류 메시지 최소화
        }

        # Train/Validation Split
        X_train, X_val, y_train, y_val = train_test_split(
            X[selected_features], y, test_size=0.2, random_state=42
        )

        # LightGBM Regressor
        model = lgb.LGBMRegressor(**param, n_estimators=1000)

        # 학습
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='rmse',
            callbacks=[lgb.early_stopping(stopping_rounds=50)]
        )

        # 검증 RMSE 계산
        preds = model.predict(X_val)
        rmse = mean_squared_error(y_val, preds, squared=False)
        return rmse

    # Optuna를 사용하여 하이퍼파라미터 최적화
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)

    # 최적 특성과 하이퍼파라미터 가져오기
    best_indices = [
        i for i in range(len(feature_names)) if study.best_params[f"use_feature_{i}"]
    ]
    best_features = [feature_names[i] for i in best_indices]
    best_params = {k: v for k, v in study.best_params.items() if not k.startswith("use_feature_")}

    # 최적 특성으로 최종 모델 학습
    X_train, X_val, y_train, y_val = train_test_split(
        X[best_features], y, test_size=0.2, random_state=42
    )
    final_model = lgb.LGBMRegressor(**best_params, n_estimators=1000)
    final_model.fit(X_train, y_train)

    return final_model, best_params, best_features


In [ ]:
# 테스트 데이터에서 Id 보관
solo_test_ids = solo_test['Id']
duo_test_ids = duo_test['Id']
squad_test_ids = squad_test['Id']
others_test_ids = others_test['Id']

In [ ]:
solo_train

In [ ]:
solo_test

In [ ]:
# Solo 데이터 준비
X_solo_train, y_solo_train, X_solo_test = prepare_data(solo_train, solo_test)

# Solo 모델 학습 및 최적화
solo_model, solo_best_params, solo_best_features = train_lgb_with_feature_selection(X_solo_train, y_solo_train)
print("Solo Best Parameters:", solo_best_params)
print("Solo Selected Features:", solo_best_features)

In [ ]:
# Duo 데이터 준비
X_duo_train, y_duo_train, X_duo_test = prepare_data(duo_train, duo_test)

# Duo 모델 학습 및 최적화
duo_model, duo_best_params, duo_best_features = train_lgb_with_feature_selection(X_duo_train, y_duo_train)
print("Duo Best Parameters:", duo_best_params)
print("Duo Selected Features:", duo_best_features)

In [ ]:
# Squad 데이터 준비
X_squad_train, y_squad_train, X_squad_test = prepare_data(squad_train, squad_test)

# Squad 모델 학습 및 최적화
squad_model, squad_best_params, squad_best_features = train_lgb_with_feature_selection(X_squad_train, y_squad_train)
print("Squad Best Parameters:", squad_best_params)
print("Squad Selected Features:", squad_best_features)


In [ ]:
# Others 데이터 준비
X_others_train, y_others_train, X_others_test = prepare_data(others_train, others_test)

# Others 모델 학습 및 최적화
others_model, others_best_params, others_best_features = train_lgb_with_feature_selection(X_others_train, y_others_train)
print("Others Best Parameters:", others_best_params)
print("Others Selected Features:", others_best_features)

In [ ]:
solo_test

In [ ]:
# 선택된 특성만 사용
X_solo_test = X_solo_test[solo_best_features]
X_duo_test = X_duo_test[duo_best_features]
X_squad_test = X_squad_test[squad_best_features]
X_others_test = X_others_test[others_best_features]

# 테스트 데이터 예측
solo_test_predictions = solo_model.predict(X_solo_test)
duo_test_predictions = duo_model.predict(X_duo_test)
squad_test_predictions = squad_model.predict(X_squad_test)
others_test_predictions = others_model.predict(X_others_test)

# 예측값과 Id 병합
solo_submission = pd.DataFrame({'Id': solo_test_ids, 'winPlacePerc': solo_test_predictions})
duo_submission = pd.DataFrame({'Id': duo_test_ids, 'winPlacePerc': duo_test_predictions})
squad_submission = pd.DataFrame({'Id': squad_test_ids, 'winPlacePerc': squad_test_predictions})
others_submission = pd.DataFrame({'Id': others_test_ids, 'winPlacePerc': others_test_predictions})

# 중복된 'Id'가 있는지 확인하고 제거
submissions = [solo_submission, duo_submission, squad_submission, others_submission]
for idx, sub in enumerate(submissions):
    if sub['Id'].duplicated().any():
        print(f"중복 발견: {idx}번째 데이터셋")
        submissions[idx] = sub.drop_duplicates(subset='Id')

# 제출 데이터 생성 및 정렬
submission = pd.concat(submissions).sort_values(by='Id').reset_index(drop=True)

# 샘플 제출 파일 로드
sample_submission = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')

# 예측값 병합: sample_submission의 'Id'를 기준으로 예측값 'winPlacePerc' 병합
final_submission = sample_submission[['Id']].merge(submission, on='Id', how='left')

# 병합 후 결측값 처리
final_submission['winPlacePerc'] = final_submission['winPlacePerc'].fillna(0.5)  # 기본값 설정

# 최종 제출 파일 저장
final_submission.to_csv('submission.csv', index=False)

print("Final submission file saved as 'submission.csv'.")


치터 잡아보기

In [ ]:
train[train['kills'] >= 40]


In [ ]:
train['headshot_rate'] = train['headshotKills'] / (train['kills'] + 1)
test['headshot_rate'] = test['headshotKills'] / (test['kills'] + 1)


In [ ]:
train[(train['headshot_rate'] >= 0.9) & (train['kills'] >= 10)]


In [ ]:
def show_distplot(column):
    plt.figure(figsize=(12, 4))
    sns.distplot(train[column], bins=50)
    plt.show()

In [ ]:
show_distplot('total_distance')


In [ ]:
print("Average longest kill distance a player achieve is {:.1f}m, 95% of them not more than {:.1f}m and a maximum distance is {:.1f}m." .format(train['total_distance'].mean(),train['total_distance'].quantile(0.95),train['total_distance'].max()))

In [ ]:
train[train['total_distance'] >= 6439]['winPlacePerc'].describe()

In [ ]:
train[train['rideDistance'] >= 30000]

In [ ]:
train[train['walkDistance'] >= 13000]

In [ ]:
# Create normalized features
train['killsNorm'] = train['kills']*((100-train['playersJoined'])/100 + 1)
train['damageDealtNorm'] = train['damageDealt']*((100-train['playersJoined'])/100 + 1)
# Create normalized features
test['killsNorm'] = test['kills']*((100-test['playersJoined'])/100 + 1)
test['damageDealtNorm'] = test['damageDealt']*((100-test['playersJoined'])/100 + 1)


In [ ]:
# playersJoined
train['playersJoined'] = train.groupby('matchId')['matchId'].transform('count')
test['playersJoined'] = test.groupby('matchId')['matchId'].transform('count')

In [ ]:
train